In [ ]:
from os.path import join, isfile
from re import match
from shutil import copy2
from time import time
import json
from unicodedata import normalize


IMAGES_DIR = join('..', 'imgs')

INPUT_DIR_IMAGES = join(IMAGES_DIR, '04.Sized')
INPUT_DIR_JSON = join(IMAGES_DIR, '06.Ordered')
OUTPUT_DIR = join(IMAGES_DIR, '07.Web')

start_s = time()

old_to_new = {}

for order in ['rgb', 'hls']:
    input_file = join(INPUT_DIR_JSON, 'by_%s.json' % order)

    with open(input_file) as json_file:
        image_names = json.load(json_file)

    image_names_clean = []
    repeat_counter = 0

    for image_name in image_names:
        if (image_name in old_to_new):
            image_name_new = old_to_new[image_name]
        else:
            image_name_parse = match(r'(MUMI-[0-9]+(-[0-9])*)([\w -].*)', image_name.replace('.-', '-'))
            image_name_new = image_name_parse.groups()[0]

        if (image_name_new in image_names_clean):
            image_name_new = "%s-%s%s%s" % (image_name_new, repeat_counter, repeat_counter, repeat_counter)
            repeat_counter += 1
            # print("%s --> %s" % (image_name, image_name_new))

        image_names_clean.append(image_name_new)
        old_to_new[image_name] = image_name_new

        in_file = join(INPUT_DIR_IMAGES, '%s.jpg' % image_name)
        out_file = join(OUTPUT_DIR, '%s.jpg' % image_name_new)

        if not isfile(out_file):
            copy2(in_file, out_file)

    output_file = join(OUTPUT_DIR, 'by_%s.json' % order)
    with open(output_file, 'w', encoding='utf8') as json_file:
        json_file.write(normalize('NFC', json.dumps(image_names_clean, ensure_ascii=False)))


print("total time: %i seconds" % (time() - start_s))


In [ ]:
for order in ['rgb', 'hls']:
    input_file = join(OUTPUT_DIR, 'by_%s.json' % order)
    cnt = 0

    with open(input_file) as json_file:
        image_names = json.load(json_file)
    
    for image_name in image_names:
        new_file = join(OUTPUT_DIR, '%s.jpg' % image_name)

        if not isfile(new_file):
            print("oooops: %s not found" % new_file)
        else:
            cnt += 1
    print("found %i files" % cnt)
